# Description 

1. get all fruits that the player can load either alone or with others
2. for each of these fruits calculate summery statistics
3. for each player make a prediciton for each of the fruits and save the result in a dictionary for each player. id fruits and prob
4. given the first order believes about the othe players, choose the next target and give teh result as training back to teh prediction for the player
5. When another player loads a fruit, train the neural network with the results by mapping it with the id. 

So the nn is trained to predict the reqard maximizing choice of a player given the choices of other players

## Notes

- reward = level_player * level_food creating a clear incentive to cooperate

# Code

### Setup

#### Requirements

In [1]:
import lbforaging
import gymnasium as gym
from lbforaging.agents import H1
import time
import numpy as np
import pandas as pd
import numpy as np

from tr_lbf_addon.lbf_gym import Lbf_Gym

#### Create the enviromente

In [2]:
# register the environment in gym

field_size = 8 # size of the game board
number_players = 5 # Number of players
max_num_food = 8 # max amount of food on the board. TODO How is teh amount of food determined?
coop_mode = False # If true, all foods will have teh max level so that all foods can only be loaded by working with other players
max_episode_steps = 50 # Number of steps until one round (episode) is terminated
sight = 0  #  How far can the agents see i presume TODO
max_player_level = 1
min_player_level = 1
max_food_level = 1
min_food_level = 1
normalize_reward = True
grid_observation = False
observe_agent_levels = True # If true, the observation will include the level of the agents
penalty = 0.0 # if the player was not the one to load the food, it gets a penalty to its reward
render_mode = "human"
full_info_mode = True

id_string = "Foraging-{0}x{0}-{1}p-{2}f{3}-v3".format(field_size, number_players, max_num_food, "-coop" if coop_mode else "")

gym.envs.registration.register(
    id=id_string,
    entry_point="lbforaging.foraging:ForagingEnv",
    kwargs={
        "players": number_players,
        "max_player_level": max_player_level,
        "min_player_level": min_player_level, 
        "max_food_level": max_food_level,
        "min_food_level": min_food_level,
        "field_size": (field_size, field_size),
        "max_num_food": max_num_food,
        "sight": sight,
        "max_episode_steps": max_episode_steps,
        "force_coop": coop_mode,
        "normalize_reward" : normalize_reward,
        "grid_observation" : grid_observation,
        "observe_agent_levels" : observe_agent_levels,
        "penalty" : penalty,
        "render_mode" : render_mode,
        "full_info_mode": full_info_mode
        },
    )

s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\envs\registration.py:642: UserWarning: WARN: Overriding environment Foraging-8x8-5p-8f-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [3]:
# define the environment. A more detailed way is discribed on https://github.com/semitable/lb-foraging
env = gym.make(id_string) # "Foraging-{GRID_SIZE}x{GRID_SIZE}-{PLAYER COUNT}p-{FOOD LOCATIONS}f{-coop IF COOPERATIVE MODE}-v0"
# render_mode is "human" per default

# reset the environment with a seed
observation, info = env.reset(seed=42)

# initialize the class
tr_marla_class = Lbf_Gym(observation[0])

s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:130: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'dict'>
  logger.warn(
s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\spaces\box.py:418: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")
s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:130: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'NoneType'>
  logger.warn(


### Tests

In [4]:
episode_over = False
step_amount = 0
while not episode_over:
    tr_marla_class.update_observation(observation[0])
    actions = tr_marla_class.agents_choose_actions()
    observation, reward, terminated, truncated, info = env.step(tuple(actions))
    # let 2 senconds pass
    env.render()
    time.sleep(0.2)
    episode_over = terminated or truncated
    step_amount += 1
env.close()
print(f"Game lasted {step_amount} steps")

s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:130: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'dict'>
  logger.warn(
s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:130: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'NoneType'>
  logger.warn(
s:\coding_projects\lbf_marl\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:245: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(
s:\coding_projects\lbf_marl\.venv\Lib\site-package

Game lasted 10 steps


In [ ]:
tr_marla_class.update_observation(observation[0])
actions = tr_marla_class.agents_choose_actions()
observation, reward, terminated, truncated, info = env.step(tuple(actions))
tr_marla_class.full_info_field

# test neural network

In [36]:

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
import pandas as pd
import numpy as np
import tensorflow as tf

In [45]:
# tets data

number_of_players = 5
fruit_level = 2
min_fruit_level = 1
max_fruit_level = 3
id = 1
ground_truth = np.array([1])

known_agents = pd.DataFrame()
known_agents["id"] = [0,1,2,3,4]
known_agents["level"] = [1,1,2,2,3]
known_agents["distance_to_fruit"] = np.array([1, 2, 3, 4, 5])
known_agents.set_index("id", inplace=True)

# normalize the levels with min max scaling
max_level = known_agents["level"].max()
min_level = known_agents["level"].min()
known_agents["level"] = (known_agents["level"] - min_level) / (max_level - min_level)

# normalize the distance to the fruit with min max scaling
max_distance = known_agents["distance_to_fruit"].max()
min_distance = known_agents["distance_to_fruit"].min()
known_agents["distance_to_fruit"] = (known_agents["distance_to_fruit"] - min_distance) / (max_distance - min_distance)

# normalize the fruit level with min max scaling
fruit_level = (fruit_level - min_fruit_level) / (max_fruit_level - min_fruit_level)

# prepare input
known_agents.sort_index(inplace=True)
agents_info = known_agents.loc[id]
known_agents.drop(id, inplace=True)
# first the level of the fruit than the agents info (level, distance to fruit) than the rest of the agents info first the level than the distance to the fruit each ordered by id
input = np.array([fruit_level] + agents_info.tolist() + known_agents["level"].tolist() + known_agents["distance_to_fruit"].tolist())
input = input.reshape(1,len(input))

In [55]:
# Create a Sequential model
model = models.Sequential()
# Fit the normalizer on the training data
model.add(layers.Input(shape=(11,)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# Train the model

loss_fn = losses.MeanSquaredError()
optimizer = optimizers.Adam()

initial_weights = model.get_weights() 

def learn(model, input, ground_truth):
    with tf.GradientTape() as tape:
        # Forward pass
        prediction = model(input, training=True)
        loss_value = loss_fn(ground_truth, prediction)
        # caclulate the gradients
        gradients = tape.gradient(loss_value, model.trainable_variables)
    # Update the weights
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return model

learn(model, input, ground_truth)
updated_weights = model.get_weights()

# chekc if the weights have been updated
for i in range(len(initial_weights)):
    if not tf.reduce_all(tf.equal(initial_weights[i], updated_weights[i])):
        print(f"Layer {i} weights have been updated.")
    else:
        print(f"Layer {i} weights have not been updated.")

Layer 0 weights have been updated.
Layer 1 weights have been updated.
Layer 2 weights have been updated.
Layer 3 weights have been updated.


1. reward ist immernoch nicht gelöst. Wenn der agent die frucht die gewinn maximierend ist wählt, berücksichtigt er nicht dass die Frucht auch von jemand anderen genommen werden kann bevor er sie erreicht. Daher muss er immer mit den nn predicten und erst evaluaten wenn er entweder die frucht hat oder die frucht weg ist. Daher muss das neuronale netz auch die belohnung vorhersagen. Der true value ist dann die belohnung von der frucht normalisiert oder 0 wenn die frucht schon weg ist. Aber wie unterscheidet man dann zwischen frucht wurde gewählt aber er ist nicht rechtzeitig hingekommen und er wollte die frucht garnicht? Muss man den nächssten reward von einem spieler vorhersagen und davon dann auf die frucht schließen? Q values sind dann wieder wichtig? Man könnte einfgach öffentlich machen welche frucht wer wählt. Das wäre das einfache. Oder man evaluiert nur die vorhersagen die vorhersage richtig warund da wo man es nicht weiß gibt man 0.5 an oder so. Zuerst das einfach edann das schwierige machen. 

2. es muss noch eine logik eingeführt werden um zu wissen welche frucht wer gewählt hat wenn jemand einen reward bekommt. Die reards sind einfach eine liste und müssen erst noch zu den agent_infos in den observations in jeder runde hinzugefügt werden. Um zu wissen welche fruit von wem gelootet wurde kan bei denen die einen reward haben geschaut werden wo sie stehen und welche fruit zuvor da stand. Dafür muss die field creation überarbeitet werden sodass das "alte" field gespeichert wird um immer vergleichen zu können. also erst rewards in player info und altes field immer speichern und dann wenn ein agent ein reward bekommt schauen wo er steht und welche fruit er genommen hat. Vielleicht muss auch nicht das alte feld gespeichert werden sondern das ganze bevor das field überschrieben wird gemacht werden. 

3. implement teh learning function into the project

4. maybe but cognition in seperate class that agents inherits. Makes things tidier
